In [1]:
import sys
import os
import numpy as np
import pandas as pd
from numpy.lib import recfunctions as rfn

import math
import tables
import pickle as pkl


import matplotlib as mpl
#Graphical options
import itertools
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
mpl.rc("savefig", dpi=300)

from icecube import dataio, dataclasses, astro
import time
import random

sys.path.append("/data/user/liruohan/dataset/OscNext")
#import TimeConverter as TConverter
import os.path

In [ ]:
#check NT format
NT_sim_file_path = '/data/ana/analyses/northern_tracks/version-005-p02/IC86_pass2_MC.npy'
NT_sim_file = np.load(NT_sim_file_path)

In [ ]:
NT_sim_file

In [ ]:
NT_sim_file['run']

In [ ]:
old_sim_file_path = '/data/user/niovine/projects/DarkMatter_OscNext/Samples/OscNext/L7/SimulationFiles/Reprocessed/OscNext_Level7_v02.00_140000_pass2_variables_NoCut.pkl'
old_sim_file = np.load(old_sim_file_path, allow_pickle=True)

In [ ]:
old_sim_file

In [ ]:
np.sum(old_sim_file['140000']['L7OscNext_bool']),len(old_sim_file['140000']['L7OscNext_bool'])

In [ ]:
plt.hist(np.rad2deg(old_sim_file['120000']['reco_TotalEnergy']),bins=np.linspace(0,500,50))
plt.hist(np.rad2deg(old_sim_file['120000']['true_Energy']),bins=np.linspace(0,500,50))

In [2]:
sim_file_folder='/data/user/niovine/projects/DarkMatter_OscNext/Samples/OscNext/L7/SimulationFiles/Reprocessed/'
pid_dict={'nu_e':'120000','nu_mu':'140000','nu_tau':'160000'}
#pid_dict={'mu':'130000'}

In [ ]:
# minimal amount of exp data: run, ra, dec, ang_err, time, log energy
# minimal amount of mc data: run, ra, dec, ang_err, time, log energy, true_ra, true_dec, true_energy, mcweight
# datafields= {
#         'run': ANALYSIS_EXP,
#         'ra': ANALYSIS_EXP,
#         'dec': ANALYSIS_EXP,
#         'ang_err': ANALYSIS_EXP,
#         'time': ANALYSIS_EXP,
#         'log_energy': ANALYSIS_EXP,
#         'true_ra': ANALYSIS_MC,
#         'true_dec': ANALYSIS_MC,
#         'true_energy': ANALYSIS_MC,
#         'mcweight': ANALYSIS_MC,
#     }

In [ ]:
test_arr=np.array([1,2,3,4,5,6,7,8,9])
test_mask = (test_arr>5)
test_arr[test_mask]

In [4]:
for name, pid in pid_dict.items():    
    
    OSC_sim_path=sim_file_folder+'OscNext_Level7_v02.00_{}_pass2_variables_NoCut.pkl'.format(pid)
    OSC_sim = np.load(OSC_sim_path,'r', allow_pickle=True)
    

    dtype_ostnext=np.dtype([('ow', '<f8'), ('trueE', '<f8'), ('prompt', '<f8'), ('astro', '<f8'),
                            ('zen', '<f8'), ('angErr', '<f8'), ('logE', '<f8'), ('dec', '<f8'),
                            ('trueDec', '<f8'), ('trueRa', '<f8'), ('trueAzi', '<f8'), ('trueZen', '<f8'),
                            ('ra', '<f8'), ('psi', '<f8'), ('run', '<i8'), ('event', '<i8'), ('subevent', '<i8'),
                            ('time', '<f8'), ('azi', '<f8'), ('conv', '<f8')])
    
    #apply cut
    #L4
    noise_l4_mask = OSC_sim[pid]['L4noise_classifier'] > 0.95
    #L5
    hitdom_l5_mask = OSC_sim[pid]['L5nHit_DOMs'] > 2.5
    #L7
    #muon_mask
    #muon_l7_mask = OSC_sim[pid]['L7muon_classifier_all'] > 0.4 #half_remain
    muon_l7_mask = OSC_sim[pid]['L7muon_classifier_up'] > 0.4 #half_remain
    #Coincident event cuts
    ntop_l7_mask = OSC_sim[pid]['L7_ntop15'] > 2.5
    nouter_l7_mask = OSC_sim[pid]['L7_nouter'] < 7.5
    #Containment cuts
    vertex_l7_mask1 = OSC_sim[pid]['L7reco_vertex_z']>-500
    vertex_l7_mask2 = OSC_sim[pid]['L7reco_vertex_z']<-200
    rho36_l7_mask = OSC_sim[pid]['L7reco_vertex_rho36'] < 300	
    #reconstructed vertex inside detector
    recotime_l7_mask = OSC_sim[pid]['L7reco_time'] < 14500
    #energy mask
    energy_mask1 = ((OSC_sim[pid]['reco_TotalEnergy'])>5) #57000 remain
    energy_mask2 = ((OSC_sim[pid]['reco_TotalEnergy'])<300)

    #
    combined_masks = ((OSC_sim[pid]['L7OscNext_bool']==1) & energy_mask1 & energy_mask2 )
    # combined_masks = ( muon_l7_mask & hitdom_l5_mask & noise_l4_mask & ntop_l7_mask 
    #                   & vertex_l7_mask1 & vertex_l7_mask2 & nouter_l7_mask & rho36_l7_mask & recotime_l7_mask)

    
    
    df_oscnext={}
    print(pid)
    # minimal amount of mc data: run, ra, dec, ang_err, time, log energy, true_ra, true_dec, true_energy, mcweight
    df_oscnext['ow']=OSC_sim[pid]['OneWeight'][combined_masks]
    df_oscnext['trueE']=OSC_sim[pid]['true_Energy'][combined_masks]
    df_oscnext['angErr']= abs(OSC_sim[pid]['true_psi']-OSC_sim[pid]['reco_psi'])[combined_masks] #?
    df_oscnext['logE']=np.log10(OSC_sim[pid]['reco_TotalEnergy'])[combined_masks]
    df_oscnext['dec']=OSC_sim[pid]['reco_Dec'][combined_masks]
    df_oscnext['trueDec']=OSC_sim[pid]['true_Dec'][combined_masks]
    df_oscnext['trueRa']=OSC_sim[pid]['true_RA'][combined_masks]
    df_oscnext['ra']=OSC_sim[pid]['reco_RA'][combined_masks]
    df_oscnext['run']=OSC_sim[pid]['PID'][combined_masks]
    df_oscnext['time']=OSC_sim[pid]['MJD_time'][combined_masks]
    df_oscnext['trueAzi']=OSC_sim[pid]['true_Azimuth'][combined_masks]
    df_oscnext['azi']=OSC_sim[pid]['reco_Azimuth'][combined_masks]
    df_oscnext['trueZen']=OSC_sim[pid]['true_Zenith'][combined_masks]
    df_oscnext['zen']=OSC_sim[pid]['reco_Zenith'][combined_masks]
    df_oscnext['psi']=OSC_sim[pid]['reco_psi'][combined_masks]
    df_oscnext['conv']=OSC_sim[pid]['AtmWeight'][combined_masks]
    #trivial field
    df_oscnext['prompt']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks]
    df_oscnext['astro']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks]
#    df_oscnext['zen']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks] #OSC_sim[pid]['true_Zenith']
#    df_oscnext['trueAzi']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks]#OSC_sim[pid]['true_Azimuth']
#    df_oscnext['trueZen']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks]#OSC_sim[pid]['true_Zenith']
#    df_oscnext['psi']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks]#OSC_sim[pid]['reco_psi']
    df_oscnext['event']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks]#OSC_sim[pid]['NEvents'][combined_masks] # ?
    df_oscnext['subevent']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks]
#    df_oscnext['azi']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks]#OSC_sim[pid]['reco_Azimuth']
#    df_oscnext['conv']=np.zeros(np.shape(OSC_sim[pid]['true_Energy']))[combined_masks]#OSC_sim[pid]['AtmFlux_nue']+OSC_sim[pid]['AtmFlux_numu']

    print(len(df_oscnext['ow']),len(df_oscnext['trueE']))
    #file to save
    save_file_name = "/data/user/liruohan/dataset/OscNext/data/pass2_Level7_v02.04_{}.npy".format(name)
    temp_array=np.vstack((df_oscnext['ow'],df_oscnext['trueE'],df_oscnext['prompt'],df_oscnext['astro'],
                          df_oscnext['zen'], df_oscnext['angErr'],df_oscnext['logE'],df_oscnext['dec'],
                          df_oscnext['trueDec'], df_oscnext['trueRa'],df_oscnext['trueAzi'],df_oscnext['trueZen'],
                          df_oscnext['ra'], df_oscnext['psi'],df_oscnext['run'],df_oscnext['event'],
                          df_oscnext['subevent'],df_oscnext['time'],df_oscnext['azi'], df_oscnext['conv']))
    temp_array_trans=np.transpose(temp_array)
    temp_array_final=rfn.unstructured_to_structured(temp_array_trans, dtype_ostnext)
    np.save(save_file_name,temp_array_final)
    print(name+'done')

120000
1480682 1480682
nu_edone
140000
3510004 3510004
nu_mudone
160000
1597669 1597669
nu_taudone
